# BUILDING APPAREL DETECTION ML MODEL USING NVIDIA’S TRANSFER LEARNING TOOLKIT (TLT)
Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where a model trained on one task is re-trained to use on a different task.

We will be using DetectNet_v2 - NVIDIA-developed object-detection model for TLT which supports subtasks such as train, prune, evaluate and export.  


## 1. Set up env variables <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.


In [ ]:
# Setting up env variables for cleaner command line commands.
%env KEY=tlt_apparels
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/detectnet_v2
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments
%env SPECS_DIR=/workspace/examples/detectnet_v2/specs
%env NUM_GPUS=1

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

Our dataset is mostly from open source images and videos. 

The data downloaded from all the sources and converted into kitti format then distributed as follows -
* training images in `$DATA_DOWNLOAD_DIR/training/image_2`
* training labels in `$DATA_DOWNLOAD_DIR/training/label_2`
* testing images in `$DATA_DOWNLOAD_DIR/testing/image_2`


### A. Verify downloaded dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
# verify kitti dataset count 
import os

DATA_DIR = os.environ.get('DATA_DOWNLOAD_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/image_2")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/label_2")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "testing/image_2")))
print("Number of images in the trainval set. {}".format(num_training_images))
print("Number of labels in the trainval set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

### B. Prepare tf records from kitti format dataset <a class="anchor" id="head-1-2"></a>

* By using tfrecords spec file(detectnet_v2_tfrecords_kitti_trainval.txt) converting it to kitti format dataset

*Note: TfRecords only need to be generated once.*

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting Tfrecords for kitti trainval dataset")
!detectnet_v2 dataset_convert \
              -d $SPECS_DIR/detectnet_v2_tfrecords_kitti_trainval.txt \
              -o $DATA_DOWNLOAD_DIR/tfrecords/kitti_trainval/kitti_trainval 

# Verify tfrecords count
!ls -rlt $DATA_DOWNLOAD_DIR/tfrecords/kitti_trainval/

### C. Download pre-trained model <a class="anchor" id="head-1-3"></a>
Download the correct pretrained model from the NGC model registry. For optimum results please download model templates from `nvidia/tlt_pretrained_detectnet_v2`. 

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tlt_pretrained_detectnet_v2:*

In [ ]:
# Create the target destination to download the model.
!mkdir -p $USER_EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tlt_pretrained_detectnet_v2:resnet18 \
    --dest $USER_EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
# Verify model is downloded to directory
!ls -rlt $USER_EXPERIMENT_DIR/pretrained_resnet18/tlt_pretrained_detectnet_v2_vresnet18


## 2. Provide training specification <a class="anchor" id="head-2"></a>
 
 Keep specification file at $SPECS_DIR/apparel_train_resnet18_kitti.txt

## 3. Run TLT training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models


In [ ]:
!detectnet_v2 train -e $SPECS_DIR/apparel_train_resnet18_kitti.txt \
                        -r $USER_EXPERIMENT_DIR/apparel_unpruned \
                        -k $KEY \
                        -n resnet18_detector \
                        --gpus $NUM_GPUS

## 4. Evaluate the trained model <a class="anchor" id="head-4"></a>

In [ ]:
!detectnet_v2 evaluate -e $SPECS_DIR/apparel_train_resnet18_kitti.txt\
                       -m $USER_EXPERIMENT_DIR/apparel_unpruned/weights/resnet18_detector.tlt \
                       -k $KEY

## 5. Prune the trained model <a class="anchor" id="head-5"></a>

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $USER_EXPERIMENT_DIR/apparel_pruned

In [ ]:
!detectnet_v2 prune \
              -m $USER_EXPERIMENT_DIR/apparel_unpruned/weights/resnet18_detector.tlt \
              -o $USER_EXPERIMENT_DIR/apparel_pruned/resnet18_nopool_bn_detectnet_v2_pruned.tlt \
              -eq union \
              -pth 0.0000052 \
              -k $KEY

## 6. Retrain the pruned model <a class="anchor" id="head-6"></a>


In [ ]:
# Retraining using the pruned model as pretrained weights 
!detectnet_v2 train -e $SPECS_DIR/apparel_retrain_resnet18_kitti.txt \
                    -r $USER_EXPERIMENT_DIR/apparel_retrain \
                    -k $KEY \
                    -n apparel_pruned \
                    --gpus $NUM_GPUS

## 7. Evaluate the retrained model <a class="anchor" id="head-7"></a>

This section evaluates the pruned and retrained model, using `tlt-evaluate`.

In [ ]:
!detectnet_v2 evaluate -e $SPECS_DIR/apparel_retrain_resnet18_kitti.txt \
                       -m $USER_EXPERIMENT_DIR/apparel_retrain/weights/resnet18_detector_pruned.tlt \
                       -k $KEY

## 8. Visualize inferences <a class="anchor" id="head-8"></a>


In [ ]:
# Running inference for detection on n images
!detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_tlt.txt \
                        -o $USER_EXPERIMENT_DIR/tlt_infer_testing \
                        -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                        -k $KEY

In [67]:
# Simple grid visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['USER_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the first 12 images.
OUTPUT_PATH = 'tlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 16 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 9. Deploy! <a class="anchor" id="head-9"></a>

In [ ]:
!mkdir -p $USER_EXPERIMENT_DIR/experiment_dir_final15
# Removing a pre-existing copy of the etlt if there has been any.
import os
output_file=os.path.join(os.environ['USER_EXPERIMENT_DIR'],
                         "apparel_final/resnet18_detector.etlt")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!detectnet_v2 export \
            -m $USER_EXPERIMENT_DIR/aparel_retrain/weights/resnet18_detector_pruned.tlt \
            -o $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector.etlt \
            -k $KEY

### A. Int8 Optimization <a class="anchor" id="head-9-1"></a>


In [ ]:
!rm -rf $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector.etlt
!rm -rf $USER_EXPERIMENT_DIR/apparel_final/calibration.bin
!detectnet_v2 export \
              -m $USER_EXPERIMENT_DIR/apparel_retrain/weights/resnet18_detector_pruned13.tlt \
              -o $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector.etlt \
              -k $KEY  \
              --cal_data_file $USER_EXPERIMENT_DIR/apparel_final/calibration.tensor \
              --data_type int8 \
              --batches 10 \
              --batch_size 4 \
              --max_batch_size 4\
              --engine_file $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector.trt.int8 \
              --cal_cache_file $USER_EXPERIMENT_DIR/apparel_final/calibration.bin \
              --verbose

### B. Generate TensorRT engine <a class="anchor" id="head-9-2"></a>


In [ ]:
!tlt-converter $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector.etlt \
               -k $KEY \
               -c $USER_EXPERIMENT_DIR/apparel_final/calibration.bin \
               -o output_cov/Sigmoid,output_bbox/BiasAdd \
               -d 3,256,256 \
               -i nchw \
               -m 64 \
               -t int8 \
               -e $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector.trt \
               -b 4

## 10. QAT workflow <a class="anchor" id="head-11"></a>
This section delves into the newly enabled Quantization Aware Training feature with DetectNet_v2. The workflow defined below converts a pruned model from section [5](#head-5). 

### A. Convert pruned model to QAT and retrain <a class="anchor" id="head-11-1"></a>


In [ ]:
!detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                    -r $USER_EXPERIMENT_DIR/apparel_retrain_qat \
                    -k $KEY \
                    -n resnet18_detector_pruned_qat \
                    --gpus $NUM_GPUS

### B. Evaluate QAT converted model <a class="anchor" id="head-11-2"></a>


In [ ]:
!detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                       -m $USER_EXPERIMENT_DIR/final_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                       -k $KEY \
                       -f tlt

### C. Export QAT trained model to int8 <a class="anchor" id="head-11-3"></a>


In [ ]:
!rm -rf $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector_qat.etlt
!rm -rf $USER_EXPERIMENT_DIR/apparel_final/calibration_qat.bin
!detectnet_v2 export \
              -m $USER_EXPERIMENT_DIR/apparel_retrain/weights/resnet18_detector_pruned_qat.tlt \
              -o $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector_qat.etlt \
              -k $KEY  \
              --data_type int8 \
              --batch_size 64 \
              --max_batch_size 64\
              --engine_file $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector_qat.trt.int8 \
              --cal_cache_file $USER_EXPERIMENT_DIR/apparel_final/calibration_qat.bin \
              --verbose

### D. Evaluate a QAT trained model using the exported TensorRT engine <a class="anchor" id="head-11-4"></a>


In [ ]:
!detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                       -m $USER_EXPERIMENT_DIR/apparel_final/resnet18_detector_qat.trt.int8 \
                       -f tensorrt

### D. Inference using QAT engine <a class="anchor" id="head-11-5"></a>


In [ ]:
!detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt_qat.txt \
                        -o $USER_EXPERIMENT_DIR/tlt_infer_testing_qat \
                        -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                        -k $KEY